<a href="https://colab.research.google.com/github/ramzihaddad12/ATMSystem/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [7]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [8]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [22]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.

# function to get the incoming nodes and the number of outgoing edges for each node
def get_incoming_list_and_outgoing_link_count(nodes, edges):
  incoming = {} # dictionary to store list of incoming nodes for each node 
  outgoing = {} # dictionary to store count of outgoing edges for each node 
  for edge in edges:
    u, v = edge
    incoming_list_for_v = incoming.get(v, [])
    incoming_list_for_v.append(u)
    incoming[v] = incoming_list_for_v

    outgoing_count = outgoing.get(u, 0)
    outgoing[u] = outgoing_count + 1

  return incoming, outgoing

# function to transform nodes list to dictionary 
def list_nodes_to_dict(nodes):
  dict_for_nodes = {}
  for node in nodes:
    num = node[0]
    letter = node[1]

    dict_for_nodes[num] = letter

  return dict_for_nodes

# function performing iterative pagerank
def page_rank_fixed_iter(nodes, edges, iterations=10):
  lambda_damping = 0.85
  num_of_nodes = len(nodes)
  incoming, outgoing_count = get_incoming_list_and_outgoing_link_count(nodes, edges)
  prev_page_rank = [None] * num_of_nodes
  curr_page_rank = [None] * num_of_nodes

  # initial setting (all nodes equally likely to being visited)
  for node in nodes:
    num_node = node[0]
    prev_page_rank[num_node] = 1 / num_of_nodes 

  for i in range(iterations):

    # finding the pagerank total for nodes/pages that have no outgoing links 
    no_outgoing_links_term = 0
    for node in nodes:
      num_node = node[0]
      if num_node not in outgoing_count:
        no_outgoing_links_term += lambda_damping * prev_page_rank[num_node] / num_of_nodes
    
    # finding the page rank for nodes/pages that have outgoing links 
    for node in nodes:
      num_node = node[0]
      curr_page_rank[num_node] = ((1 - lambda_damping) / num_of_nodes) + no_outgoing_links_term

      if num_node in incoming:
        incoming_nodes = incoming.get(num_node)

        for incoming_node in incoming_nodes:
          curr_page_rank[num_node] += lambda_damping * prev_page_rank[incoming_node] / outgoing_count[incoming_node]
    
    # refresh iteration
    prev_page_rank = curr_page_rank

  # prettify the result
  result = []
  dict_for_nodes = list_nodes_to_dict(nodes)
  for i in range(len(curr_page_rank)):
    result.append((i, dict_for_nodes[i], curr_page_rank[i]))

  return result

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
page_rank_fixed_iter(small_nodes, small_edges, 1)
page_rank_fixed_iter(small_nodes, small_edges, 10)
page_rank_fixed_iter(small_nodes, small_edges, 100)

[(0, 'A', 0.2521271053751956),
 (1, 'B', 0.1393061853185386),
 (2, 'C', 0.1513064898667053),
 (3, 'D', 0.11890782257353921),
 (4, 'E', 0.18704590699931614),
 (5, 'F', 0.1513064898667053)]

## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [9]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2022-10-07 23:01:22--  https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  20.7MB/s    in 0.2s    

2022-10-07 23:01:22 (20.7 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2022-10-07 23:01:23--  https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12829526 (12M) [application/x-gzip]
Saving to: ‘edges-edu.txt.gz’

edges-edu.txt.gz    100%[===================>]  12.23M  57.9MB/s    in 0.2s    

2022-10-07 23:01:23 (57.9 MB/s) - ‘edges-edu.txt.gz’ saved [12829526/128295

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [42]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
            
EDGES_FILE_NAME = '/content/edges-edu.txt'
VERTICES_FILE_NAME = '/content/vertices-edu.txt'

edges = []
nodes = []
with open(EDGES_FILE_NAME) as f:
    edges_list = list(f)
with open(VERTICES_FILE_NAME) as f:
    nodes_list = list(f)
    
for edge_list in edges_list:
    split_line = edge_list.split()
    edges.append((int(split_line[0]), int(split_line[1])))

for node_list in nodes_list:
    split_line = node_list.split()
    nodes.append((int(split_line[0]), split_line[1]))


Mounted at /content/drive
[(1, 'edu.06hxbt'), (2, 'edu.082ifc'), (3, 'edu.083mjs'), (4, 'edu.09xzrr'), (5, 'edu.0aoqqj'), (6, 'edu.0ax4el'), (7, 'edu.0c5fez'), (8, 'edu.0cosn2'), (9, 'edu.0dcdp8')]


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [ ]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

def page_rank(nodes, edges, threshold=1):
  return []

# Run until perplexity changes by less than 1
PR = page_rank(nodes, edges, 1)

## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [ ]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.

Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

# - the proportion of websites with no out-links (i.e., sink nodes);

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
